In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
format_df = "%Y-%m-%d %H:%M:%S"

In [17]:
symbol = "EURUSD"
file = f"{symbol} MTTF Magnitude Vector Backtest Log.csv"

data = pd.read_csv(file)


In [18]:
def detect_year(x):
    format_df = "%Y-%m-%d %H:%M:%S"
    return datetime.strptime(x, format_df).year

In [19]:
def calculate_drawdown(total_R):
    global max_dd, mean_dd, median_dd
    
    R_array = np.array(total_R).cumsum()

    peak = np.maximum.accumulate(R_array)
    dd = R_array - peak
    
    max_dd = round(dd.min(),3)
    mean_dd = round(dd.mean(),3)
    median_dd = round(np.percentile(dd, 50),3)
    
    return max_dd, mean_dd, median_dd

### Delete Present Files

In [5]:
maintain_files = [f"{symbol} MTTF Magnitude Vector Backtest Log.csv",
                  '.ipynb_checkpoints',
                 'Backtest Data Analysis.ipynb',
                 'Backtesting Framework- Magnitude Vector.ipynb' ,
                f'MV Strategy Summary for {symbol}.png',
                    f'{symbol} M5 DATA 2018-NOW.csv']


In [6]:
for file in os.listdir():
    if file not in maintain_files:
        os.remove(file)
        print(file, "REMOVED")

GBPUSD Expectancy Metrics.json REMOVED
GBPUSD_2018_btest_log.csv REMOVED
GBPUSD_2019_btest_log.csv REMOVED
GBPUSD_2020_btest_log.csv REMOVED
GBPUSD_2021_btest_log.csv REMOVED
GBPUSD_2022_btest_log.csv REMOVED
GBPUSD_2023_btest_log.csv REMOVED
GBPUSD_2024_btest_log.csv REMOVED
GBPUSD_2025_btest_log.csv REMOVED
GBPUSD_session_metrics.json REMOVED


### Expectancy Per Year

#### Separate by Year

In [7]:
detect_year(data["Datetime"].iloc[1201])

2024

In [8]:
years = range(2018, 2026, 1)
for year in years:

    file = f"{symbol}_{year}_btest_log.csv"
    
    for idx, row in data.iterrows():
        if detect_year(row["Datetime"]) == year:
            new_data = data.iloc[idx].to_dict()
            new_data = pd.DataFrame([new_data])
            
            try:

                log = pd.read_csv(file)
                log = pd.concat([log, new_data], axis = 0)
                log.to_csv(file, mode = "w", index = False)
                print(f"{year} LOG UPDATED")                    

            except FileNotFoundError:
                new_data.to_csv(file, mode = "w", index  =False)
                print(f"{year} LOG UPDATED")

    

2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDATED
2018 LOG UPDAT

#### Calculating Expectancy by Year

In [9]:
years = range(2018, 2026, 1)
for year in years:
    file = f"{symbol}_{year}_btest_log.csv"

    

In [10]:
import json
json_file = f"{symbol} Expectancy Metrics.json"
with open(json_file, "w") as f:
    data = []
    json.dump(data, f)
    
for year in years:

        file = f"{symbol}_{year}_btest_log.csv"
        
        if os.path.exists(file):
        
            df = pd.read_csv(file)
            P_win = (df["Trade Result"].value_counts() / len(df)).values[0]
            P_loss = (df["Trade Result"].value_counts() / len(df)).values[1]
            
            total_R = []
            win_R = []
            loss_R = []
            
            for idx, row in df.iterrows():
                if row["Trade Result"] == 1:
                    total_R.append(row["win rr"])
                    win_R.append(row["win rr"])
                if row["Trade Result"] == 0:
                    total_R.append(-row["loss rr"])
                    loss_R.append(row["loss rr"])      
            
            ### Expectancy = (P_win ×R_w)−(P_loss×R_l)
            
            E = (P_win * np.mean(win_R)) - (P_loss * np.mean(loss_R))
            avg_R = np.mean(total_R)

            calculate_drawdown(total_R)
            
            metrics = {f"{year}" : {"trades" : len(df),
                                 "p_win" : P_win,
                                 "p_loss" : P_loss,
                                 "expectancy" : round(E, 4),
                                 "var_R" : round(np.var(total_R),4),
                                 "std_R" : round(np.std(total_R), 4),
                                 "mean_R" : round(avg_R, 4),
                                 "cum_R" : np.array(total_R).sum(),
                                 "sharpe" : E/(np.std(total_R)),
                                    "max_dd" : max_dd,
                                    "mean_dd" : mean_dd,
                                    "median_dd" : median_dd
                                                 }
                       
                    }
            
            with open(json_file, "r") as f:
                data = json.load(f)
                if data is not None:
                    data.append(metrics)
                    with open(json_file, "w") as f:
                        json.dump(data, f)
                        print(f"{year} JSON UPDATED")


2018 JSON UPDATED
2019 JSON UPDATED
2020 JSON UPDATED
2021 JSON UPDATED
2022 JSON UPDATED
2023 JSON UPDATED
2024 JSON UPDATED
2025 JSON UPDATED


### Expectancy by Session

In [11]:
file = f"{symbol} MTTF Magnitude Vector Backtest Log.csv"

data = pd.read_csv(file)

In [12]:
sessions = ["US","EU", "ASIAN"]
file = f"{symbol}_session_metrics.json"

with open(file, "w") as f:
    empty_list = []
    json.dump(empty_list, f)

for session in sessions:
    df = data[data["Session"] == session]

    P_win = (df["Trade Result"].value_counts() / len(df)).values[0]
    P_loss = (df["Trade Result"].value_counts() / len(df)).values[1]
    
    total_R = []
    win_R = []
    loss_R = []
    
    for idx, row in df.iterrows():
        if row["Trade Result"] == 1:
            total_R.append(row["win rr"])
            win_R.append(row["win rr"])
        if row["Trade Result"] == 0:
            total_R.append(-row["loss rr"])
            loss_R.append(row["loss rr"])      
    
    ### Expectancy = (P_win ×R_w)−(P_loss×R_l)
    
    E = (P_win * np.mean(win_R)) - (P_loss * np.mean(loss_R))
    avg_R = np.mean(total_R)

    calculate_drawdown(total_R)
    
    metrics = {f"{session}" : {"trades" : len(df),
                         "p_win" : P_win,
                         "p_loss" : P_loss,
                         "expectancy" : round(E, 4),
                         "var_R" : round(np.var(total_R),4),
                         "std_R" : round(np.std(total_R), 4),
                         "mean_R" : round(avg_R, 4),
                         "cum_R" : np.array(total_R).sum(),
                         "sharpe" : E/(np.std(total_R)),
                           "max_dd" : max_dd,
                            "mean_dd" : mean_dd,
                           "median_dd" : median_dd
                                         }
               
            }
    file = f"{symbol}_session_metrics.json"
    
    with open(file, "r") as f:
        new_data = json.load(f)
        if new_data is not None:
            new_data.append(metrics)
            with open(file, "w") as f:
                json.dump(new_data, f)
                print(f"{session} JSON UPDATED")    

US JSON UPDATED
EU JSON UPDATED
ASIAN JSON UPDATED


### Calculating Overall Expectancy

In [13]:
json_file = f"{symbol}_OVERALL_RESULTS COVID INCLUDED.json"


file = f"{symbol} MTTF Magnitude Vector Backtest Log.csv"

df = pd.read_csv(file)

P_win = (df["Trade Result"].value_counts() / len(df)).values[0]
P_loss = (df["Trade Result"].value_counts() / len(df)).values[1]

total_R = []
win_R = []
loss_R = []

for idx, row in df.iterrows():
    if row["Trade Result"] == 1:
        total_R.append(row["win rr"])
        win_R.append(row["win rr"])
    if row["Trade Result"] == 0:
        total_R.append(-row["loss rr"])
        loss_R.append(row["loss rr"])      

### Expectancy = (P_win ×R_w)−(P_loss×R_l)

E = (P_win * np.mean(win_R)) - (P_loss * np.mean(loss_R))
avg_R = np.mean(total_R)

calculate_drawdown(total_R)

metrics = {"trades" : len(df),
             "p_win" : P_win,
             "p_loss" : P_loss,
             "expectancy" : round(E, 4),
             "var_R" : round(np.var(total_R),4),
             "std_R" : round(np.std(total_R), 4),
             "mean_R" : round(avg_R, 4),
             "cum_R" : np.array(total_R).sum(),
             "sharpe" : E/(np.std(total_R)),
            "max_dd" : max_dd,
           "mean_dd" : mean_dd,
           "median_dd" : median_dd
             }

with open(json_file, "w") as f:
    json.dump(metrics, f)
    

In [14]:

json_file = f"{symbol}_OVERALL_RESULTS COVID EXCLUDED.json"


file = f"{symbol} MTTF Magnitude Vector Backtest Log.csv"

df = pd.read_csv(file)

#P_win = (df["Trade Result"].value_counts() / len(df)).values[0]
#P_loss = (df["Trade Result"].value_counts() / len(df)).values[1]

total_R = []
win_R = []
loss_R = []

win_trade = []
loss_trade= []

years = [2018, 2019, 2021, 2022, 2023, 2024, 2025]
for idx, row in df.iterrows():
    if detect_year(row["Datetime"]) in years:
        
        if row["Trade Result"] == 1:
            total_R.append(row["win rr"])
            win_R.append(row["win rr"])
            win_trade.append(1)
        if row["Trade Result"] == 0:
            total_R.append(-row["loss rr"])
            loss_R.append(row["loss rr"])  
            loss_trade.append(1)
    
    ### Expectancy = (P_win ×R_w)−(P_loss×R_l)

P_win = len(win_trade) / (len(win_trade) + len(loss_trade))
P_loss = len(loss_trade) / (len(win_trade) + len(loss_trade))


E = (P_win * np.mean(win_R)) - (P_loss * np.mean(loss_R))
avg_R = np.mean(total_R)

calculate_drawdown(total_R)

metrics = {"trades" : len(total_R),
             "p_win" : P_win,
             "p_loss" : P_loss,
             "expectancy" : round(E, 4),
             "var_R" : round(np.var(total_R),4),
             "std_R" : round(np.std(total_R), 4),
             "mean_R" : round(avg_R, 4),
             "cum_R" : np.array(total_R).sum(),
             "sharpe" : E/(np.std(total_R)),
            "max_dd" : max_dd,
           "mean_dd" : mean_dd,
           "median_dd" : median_dd
             }

with open(json_file, "w") as f:
    json.dump(metrics, f)

In [20]:
### Trimming Session

In [21]:
json_file = f"{symbol}_OVERALL_RESULTS WITHOUT ASIAN SESSION.json"
import json

file = f"{symbol} MTTF Magnitude Vector Backtest Log.csv"

df = pd.read_csv(file)

#P_win = (df["Trade Result"].value_counts() / len(df)).values[0]
#P_loss = (df["Trade Result"].value_counts() / len(df)).values[1]

total_R = []
win_R = []
loss_R = []

win_trade = []
loss_trade= []
NO_SESSION = ["ASIAN"]
for idx, row in df.iterrows():
    
    if row["Session"] not in NO_SESSION:
        
        if row["Trade Result"] == 1:
            total_R.append(row["win rr"])
            win_R.append(row["win rr"])
            win_trade.append(1)
        if row["Trade Result"] == 0:
            total_R.append(-row["loss rr"])
            loss_R.append(row["loss rr"])  
            loss_trade.append(1)
    
    ### Expectancy = (P_win ×R_w)−(P_loss×R_l)

P_win = len(win_trade) / (len(win_trade) + len(loss_trade))
P_loss = len(loss_trade) / (len(win_trade) + len(loss_trade))


E = (P_win * np.mean(win_R)) - (P_loss * np.mean(loss_R))
avg_R = np.mean(total_R)

calculate_drawdown(total_R)

metrics = {"trades" : len(total_R),
             "p_win" : P_win,
             "p_loss" : P_loss,
             "expectancy" : round(E, 4),
             "var_R" : round(np.var(total_R),4),
             "std_R" : round(np.std(total_R), 4),
             "mean_R" : round(avg_R, 4),
             "cum_R" : np.array(total_R).sum(),
             "sharpe" : E/(np.std(total_R)),
            "max_dd" : max_dd,
           "mean_dd" : mean_dd,
           "median_dd" : median_dd
             }

with open(json_file, "w") as f:
    json.dump(metrics, f)